In [7]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model,  Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16
from keras.layers import Lambda, Cropping2D, Activation, ELU
from keras.models import model_from_json

%matplotlib inline

In [2]:
from directkeys import PressKey,ReleaseKey, W, A, S, D
from utils import *

In [19]:
t_time = 0.11

def straight():
##    if random.randrange(4) == 2:
##        ReleaseKey(W)
##    else:
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)
    
def left():
    PressKey(W)
    PressKey(A)
    #ReleaseKey(W)
    ReleaseKey(D)
    #ReleaseKey(A)
    time.sleep(t_time)
    ReleaseKey(A)
    #ReleaseKey(W)

def right():
    PressKey(W)
    PressKey(D)
    ReleaseKey(A)
    #ReleaseKey(W)
    #ReleaseKey(D)
    time.sleep(t_time)
    ReleaseKey(D)
    #ReleaseKey(W)

In [4]:
results_path = 'D:/ML_DATA/wheelai/gtaV/results/'

In [5]:
#import h5py
#f = h5py.File(model_path+'vgg_ft.h5', 'r+')
#del f['optimizer_weights']
#f.close()

In [ ]:
filter_size = 3
pool_size = (2,2)


def get_model():
    model = Sequential()
    model.add(Cropping2D(cropping=((70,0), (0,0)), input_shape=(160, 320, 3)))
    model.add(Lambda(lambda x: x/255.-0.5))
    model.add(Conv2D(3,1,1,
                        border_mode='valid',
                        name='conv0', init='he_normal'))
    model.add(Conv2D(32,filter_size,filter_size,
                        border_mode='valid',
                        name='conv1', init='he_normal'))
    model.add(ELU())
    model.add(Conv2D(32,filter_size,filter_size,
                        border_mode='valid',
                        name='conv2', init='he_normal'))
    model.add(ELU())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.5))
    model.add(Conv2D(64,filter_size,filter_size,
                        border_mode='valid',
                        name='conv3', init='he_normal'))
    model.add(ELU())

    model.add(Conv2D(64,filter_size,filter_size,
                        border_mode='valid',
                        name='conv4', init='he_normal'))
    model.add(ELU())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.5))
    model.add(Conv2D(128,filter_size,filter_size,
                        border_mode='valid',
                        name='conv5', init='he_normal'))
    model.add(ELU())
    model.add(Conv2D(128,filter_size,filter_size,
                        border_mode='valid',
                        name='conv6', init='he_normal'))
    model.add(ELU())
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(512,name='hidden1', init='he_normal'))
    model.add(ELU())
    model.add(Dropout(0.5))
    model.add(Dense(64,name='hidden2', init='he_normal'))
    model.add(ELU())
    model.add(Dropout(0.5))
    model.add(Dense(16,name='hidden3',init='he_normal'))
    model.add(ELU())
    model.add(Dropout(0.5))
    model.add(Dense(3, name='output', init='he_normal'))
    return model

In [ ]:
model = get_model()
model.load_weights(results_path+'nvidia_aug.h5')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='categorical_crossentropy')

In [8]:
model_path = results_path+'vgg_ft.json'
filepath2 = results_path+'vgg_ft.h5'

json_file = open(model_path, 'r')
vgg_model_json = json_file.read()
json_file.close()
vgg_model = model_from_json(vgg_model_json)
# load weights into new model
vgg_model.load_weights(filepath2)
print("Loaded model from disk")

Loaded model from disk


In [ ]:
#vgg_model = load_model(model_path+'vgg_ft.h5')

In [9]:
vgg_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=2e-5), metrics=['accuracy'])

#image = grab_screen(region=(10, 450, 1500, 300))
path00 = 'C:/ML_DATA/wheelai/kerakart/valid/right/right1.jpg'
img = cv2.imread(path00)
img = cv2.resize(img,(80,160))
x = image.img_to_array(img)

In [16]:
def main():
    count_down(5)
    paused = False
    while(True):

        if not paused:
            
            img = grab_screen(region=(0, 100, 1280, 640))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img,(640, 470))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            action = vgg_model.predict(x, batch_size=1,verbose=0)[0]
            
            print(action)
            max_value = max(action)
            max_index = action.index(max_value)
            

            if max_index == 1:
                left()
            elif max_index == 2:
                right()
            else:
                straight()

        keys = key_check()
        if 'T' in keys:
            if paused:
                paused = False
                print('unpaused!')
                time.sleep(1)
            else:
                print('Pausing!')
                paused = True
                ReleaseKey(A)
                ReleaseKey(W)
                ReleaseKey(D)
                time.sleep(1)


In [ ]:
main()

5
4
3
2
1
